In [2]:
%load_ext cython

In [3]:
%%cython
def cyfib(int n):
    cdef int a, b, i
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a

In [4]:
cyfib(3)

5

In [12]:

bb=b'these'

In [13]:
memv = memoryview(bb)

In [14]:
memv

In [17]:
print(memv[0])

116


In [21]:
memv[:1]

In [22]:
memv[0]='f'

TypeError: cannot modify read-only memory

In [24]:
chr(116)

't'

In [25]:
# make a byte array

In [27]:
ba = bytearray(b'if this is the casemut')

In [28]:
mutable1 = memoryview(ba)

In [29]:
mutable1

In [30]:
mutable1[:1]

In [44]:
# single indexing is not allowed, use slice

In [42]:
mutable1[0:1]=b"c"

In [43]:
ba

bytearray(b'cf this is the casemut')

In [35]:
m = memoryview('abc'.encode())

In [37]:
m[0]

97

In [45]:
# numpy and memoryview

In [46]:
import numpy as np

In [48]:
s = np.ones((10, 20, 30))

In [50]:
s.shape

(10, 20, 30)

In [51]:
np_mv =memoryview(s)

In [52]:
np_mv.shape

(10, 20, 30)

In [53]:
np_mv.ndim

3

In [55]:
# chec the data storage 

In [54]:
np_mv.strides

(4800, 240, 8)

In [56]:
# check underlying data type

In [57]:
np_mv.format

'd'

# working with numpy

### working with memory view, bad example, version-1

In [8]:
import numpy as np

In [9]:
%%cython
def summer(double[:] mv):
    cdef double d, ss = 0.0
    # This is a bad example, python treats mv as python iterator
    # calling in Python/C API for each access, slower..
    for d in mv:
        ss += d
    return ss

In [10]:
#%%cython
def summer_python(mv):
    #cdef double d, ss = 0.0
    ss = 0
    for d in mv:
        ss += d
    return ss

In [11]:
arr = np.ones((10**6,), dtype=np.double)

In [67]:
%timeit summer(arr)

94 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
%timeit summer2(arr)

61 ms ± 886 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Working with memory view, better, version-2

In [79]:
%%cython
def summer2(double[:] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    # When the Python iterator is removed, the Python/C API is not used
    # isntead, indexes into the underlying buffer
    for i in range(N):
        ss += mv[i]
    return ss

In [73]:
%timeit summer3(arr)

598 µs ± 21.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Working with memory view, version-3

In [83]:
%%cython
from cython cimport boundscheck, wraparound

#%%cython
def summer3(double[:] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    
    with boundscheck(False), wraparound(False):
        for i in range(N):
            ss += mv[i]
    return ss

In [85]:
%timeit summer3(arr)

538 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [86]:
%%cython
from cython cimport boundscheck, wraparound

#%%cython
@boundscheck(False)
@wraparound(False)
def summer3(double[:] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    
    for i in range(N):
        ss += mv[i]
    return ss

In [87]:
%timeit summer3(arr)

536 µs ± 3.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [88]:
# compare with python numpy
%timeit arr.sum()

287 µs ± 6.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Test Fortran and C type

In [106]:
# what does stride mean?
# it actually means, the address difference between you index
# suppose, we have a 10 * 20 matrix, if you look at data[0][:], data[1][:]
# then you will note the difference is 4, as the below example shows
# In comparing, C order, will give you 80, why? Since, each row, you have 20 integers(4 btye for each)

In [95]:
mv = np.ones((10, 20), dtype=np.int32, order='F')

In [96]:
mv.strides

(4, 40)

In [104]:
mv = np.ones((10, 20), dtype=np.int32, order='C')

In [105]:
mv.strides

(80, 4)

In [89]:
mv = np.ones((10, 20, 30), dtype=np.int32, order='F')

In [92]:
mv.strides

(4, 40, 800)

In [93]:
mv = np.ones((10, 20, 30), dtype=np.int32, order='C')

In [94]:
mv.strides

(2400, 120, 4)

#### Define a typed memoryview (Cython

In [111]:
# define a c continous memory view, with the last colon ::1
# if it's a Fortran mv, then ::2, see example below

In [124]:
%%cython
cdef float[:, ::1] c_contig_mv

In [125]:
c_contig_mv = np.ones((3, 4), dtype=np.float32)

In [127]:
c_contig_mv.strides

(16, 4)

In [114]:
# define a Fortran type, or column stride mv

In [129]:
%%cython
cdef float[:, ::2] f_contig_mv
# wrong example!!! dont do this


Error compiling Cython file:
------------------------------------------------------------
...
cdef float[:, ::2] f_contig_mv
               ^
------------------------------------------------------------

C:\Users\Administrator\.ipython\cython\_cython_magic_18bc0588e7e6975dc41cf91f4692e0e4.pyx:1:16: Step must be omitted, 1, or a valid specifier.


In [130]:
%%cython
cdef float[::1, :] f_contig_mv

In [131]:
f_np = np.ones((3, 4), dtype=np.float64, order='F')
f_contig_mv = f_np

In [132]:
f_contig_mv.strides

(8, 24)

In [133]:
c_contig_mv = f_np

In [134]:
c_contig_mv.strides

(8, 24)

In [136]:
# interesting
f_contig_mv is c_contig_mv

True

## Interact with np array, pandas

In [36]:
%%cython
import numpy as np
#import pandas as pd
#obj = pd.Series({'a':[1,2,3], 'b':[10.0, 11.0, 12.0]})
obj = np.array([[1,2,3],[4,5,6]], dtype=np.int32)
cdef int[:, :] mv = obj
print(mv[0, 1])
print(mv[0,-1])
mv[0, 1] = 10
print(mv[0, 1])

In [3]:
%%cython
import pandas as pd
import numpy as np
#import pandas as pd
#obj = pd.Series({'a':[1,2,3], 'b':[10.0, 11.0, 12.0]})
obj = pd.DataFrame([[1,2,3],[4,5,6]], dtype=np.int32)
print(obj)
cdef int[:, :] mv = obj.values
print(mv[0, 1])
print(mv[0,-1])
mv[0, 1] = 10
print(mv[0, 1])

   0  1  2
0  1  2  3
1  4  5  6
2
3
10


# Two examples I cannot understand

## Example-1

In [19]:
%%cython
import pandas as pd
import numpy as np
#import pandas as pd
#obj = pd.Series({'a':[1,2,3], 'b':[10.0, 11.0, 12.0]})
obj = pd.DataFrame([[1,2,3],[4,5,6]], dtype=np.int32)
print(obj)
cdef int[:, :] mv 
mv = obj.values
cdef int[:, :] mv2 = mv[...]

#print(mv[0, 1])
#print(mv[0,-1])
mv[0, 1] = 10
print(mv[0, 1])
print(mv2[0,1])
print(id(mv))
print(id(mv2))

   0  1  2
0  1  2  3
1  4  5  6
10
10
2316057988416
2316057988416


## Example-2

In [29]:
%%cython
import numpy as np

cdef int[:, :, :] to_view, from_view
to_view = np.zeros((20, 15, 30), dtype=np.intc)
from_view = np.ones((20, 15, 30), dtype=np.intc)

# copy the elements in from_view to to_view
to_view[...] = from_view
# or
to_view[:] = from_view
# or
to_view[:, :, :] = from_view

from_view[0, 0, 0] = 999
print(to_view[0,0,0])
print(from_view[0,0,0])
print(id(to_view))
print(id(from_view))

1
999
2316095320960
2316095320960


## Example-3 from official website

https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#:~:text=Memory%20views%20can%20be%20copied%20in%20place%3A%20import,from_view%20%23%20or%20to_view%5B%3A%2C%20%3A%2C%20%3A%5D%20%3D%20from_view

In [15]:
%%cython
from cython.view cimport array as cvarray
import numpy as np

# Memoryview on a NumPy array
narr = np.arange(27, dtype=np.dtype("i")).reshape((3, 3, 3))
cdef int [:, :, :] narr_view = narr

# Memoryview on a C array
cdef int carr[3][3][3]
cdef int [:, :, :] carr_view = carr

# Memoryview on a Cython array
cyarr = cvarray(shape=(3, 3, 3), itemsize=sizeof(int), format="i")
cdef int [:, :, :] cyarr_view = cyarr

# Show the sum of all the arrays before altering it
print("NumPy sum of the NumPy array before assignments: %s" % narr.sum())

# We can copy the values from one memoryview into another using a single
# statement, by either indexing with ... or (NumPy-style) with a colon.
carr_view[...] = narr_view
cyarr_view[:] = narr_view
# NumPy-style syntax for assigning a single value to all elements.
narr_view[:, :, :] = 3

# Just to distinguish the arrays
carr_view[0, 0, 0] = 100
cyarr_view[0, 0, 0] = 1000

# Assigning into the memoryview on the NumPy array alters the latter
print("NumPy sum of NumPy array after assignments: %s" % narr.sum())

# A function using a memoryview does not usually need the GIL
cpdef int sum3d(int[:, :, :] arr) nogil:
    cdef size_t i, j, k, I, J, K
    cdef int total = 0
    I = arr.shape[0]
    J = arr.shape[1]
    K = arr.shape[2]
    for i in range(I):
        for j in range(J):
            for k in range(K):
                total += arr[i, j, k]
    return total

# A function accepting a memoryview knows how to use a NumPy array,
# a C array, a Cython array...
print("Memoryview sum of NumPy array is %s" % sum3d(narr))
print("Memoryview sum of C array is %s" % sum3d(carr))
print("Memoryview sum of Cython array is %s" % sum3d(cyarr))
# ... and of course, a memoryview.
print("Memoryview sum of C memoryview is %s" % sum3d(carr_view))

NumPy sum of the NumPy array before assignments: 351
NumPy sum of NumPy array after assignments: 81
Memoryview sum of NumPy array is 81
Memoryview sum of C array is 451
Memoryview sum of Cython array is 1351
Memoryview sum of C memoryview is 451
